<a href="https://colab.research.google.com/github/disha2sinha/Movie-Recommendation-System/blob/master/Overview_of_the_Datasets_Data_Wrangling_and_Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Libraries:**

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
from ast import literal_eval
import matplotlib.pyplot as plt
import seaborn as sns


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

**IMPORTING 'movies_metadata.csv' :**

In [ ]:
link1='https://drive.google.com/file/d/1xjASvbR3qrZ_Xx1rwmb2YoTGt081ih0G/view?usp=sharing'

In [ ]:
id='1xjASvbR3qrZ_Xx1rwmb2YoTGt081ih0G'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('movies_metadata.csv')  
movies_metadata = pd.read_csv('movies_metadata.csv',engine='python')
movies_metadata.head(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


**Exploring movies_metadata.csv :** 
Showing Shape and Column of the **movies_metadata** dataset.

In [ ]:
movies_metadata.shape

(45466, 24)

In [ ]:
movies_metadata.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

**Modifying the columns 'genre' , 'production_companies' , 'production_countries' , 'spoken_languages' :** Keeping only the value of the 'name' key in the list of these columns for ease of understanding and analysis.

In [ ]:
movies_metadata['genres'] = movies_metadata['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
movies_metadata['production_companies'] = movies_metadata['production_companies'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
movies_metadata['production_countries'] = movies_metadata['production_countries'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
movies_metadata.head(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,[Pixar Animation Studios],[United States of America],1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[Warner Bros., Lancaster Gate]",[United States of America],1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[Twentieth Century Fox Film Corporation],[United States of America],1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[Sandollar Productions, Touchstone Pictures]",[United States of America],1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [ ]:
def try_join(l):
  try:
    return '|'.join(map(str,l))
  except TypeError:
    return 'Unknown'
movies_metadata['genres']=[try_join(l)for l in movies_metadata['genres']]
movies_metadata['production_companies']=[try_join(l)for l in movies_metadata['production_companies']]
movies_metadata['production_countries']=[try_join(l)for l in movies_metadata['production_countries']]
movies_metadata.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,Animation|Comedy|Family,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,Pixar Animation Studios,United States of America,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,Adventure|Fantasy|Family,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,TriStar Pictures|Teitler Film|Interscope Commu...,United States of America,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,Romance|Comedy,NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,Warner Bros.|Lancaster Gate,United States of America,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,Comedy|Drama|Romance,NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,Twentieth Century Fox Film Corporation,United States of America,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,Comedy,NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,Sandollar Productions|Touchstone Pictures,United States of America,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


**Removing unnecessary columns from the dataset :**  The **'belongs_to_collection'** column contains the movie series in which the movie belongs along with some other irrelevant informations. So let's drop this column. Let's drop the **'poster_path'** , **'video'** , **'imdb_id'** and **'homepage'** column as well because we won't require it in our analysis.

In [ ]:
movies_metadata.drop('belongs_to_collection',inplace=True,axis=1)
movies_metadata.drop('poster_path',inplace=True,axis=1)
movies_metadata.drop('homepage',inplace=True,axis=1)
movies_metadata.drop('video',inplace=True,axis=1)
movies_metadata.drop('imdb_id',inplace=True,axis=1)
movies_metadata.head(5)

,adult,budget,genres,id,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,False,30000000,Animation|Comedy|Family,862,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0
1,False,65000000,Adventure|Fantasy|Family,8844,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,TriStar Pictures|Teitler Film|Interscope Commu...,United States of America,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0
2,False,0,Romance|Comedy,15602,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,Warner Bros.|Lancaster Gate,United States of America,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0
3,False,16000000,Comedy|Drama|Romance,31357,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,Twentieth Century Fox Film Corporation,United States of America,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0
4,False,0,Comedy,11862,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,Sandollar Productions|Touchstone Pictures,United States of America,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0


**Exploring remaining Columns :** 

In [ ]:
movies_metadata[movies_metadata['original_title'] != movies_metadata['title']][['title', 'original_title']].head(10)

,title,original_title
28,The City of Lost Children,La Cité des Enfants Perdus
29,Shanghai Triad,摇啊摇，摇到外婆桥
32,Wings of Courage,"Guillaumet, les ailes du courage"
57,The Postman,Il postino
58,The Confessional,Le confessionnal
67,French Twist,Gazon maudit
72,Les Miserables,Les misérables
79,The White Balloon,بادکنک سفید
81,Antonia's Line,Antonia
104,Nobody Loves Me,Keiner liebt mich


The **'original_title'** refers to the title of the movie in the native language in which the movie was shot.So let's drop this column too.

In [ ]:
movies_metadata.drop('original_title',inplace=True,axis=1)
movies_metadata.head(5)

,adult,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,False,30000000,Animation|Comedy|Family,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0
1,False,65000000,Adventure|Fantasy|Family,8844,en,When siblings Judy and Peter discover an encha...,17.015539,TriStar Pictures|Teitler Film|Interscope Commu...,United States of America,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0
2,False,0,Romance|Comedy,15602,en,A family wedding reignites the ancient feud be...,11.7129,Warner Bros.|Lancaster Gate,United States of America,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0
3,False,16000000,Comedy|Drama|Romance,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,Twentieth Century Fox Film Corporation,United States of America,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0
4,False,0,Comedy,11862,en,Just when George Banks has recovered from his ...,8.387519,Sandollar Productions|Touchstone Pictures,United States of America,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0


In [ ]:
movies_metadata['adult'].unique()

array(['False', 'True', ' - Written by Ørnås',
       ' Rune Balot goes to a casino connected to the October corporation to try to wrap up her case once and for all.',
       ' Avalanche Sharks tells the story of a bikini contest that turns into a horrifying affair when it is hit by a shark avalanche.'],
      dtype=object)

In [ ]:
movies_metadata[movies_metadata['adult'] ==' Rune Balot goes to a casino connected to the October corporation to try to wrap up her case once and for all.' ][['title', 'adult']].head()

,title,adult
29503,None,Rune Balot goes to a casino connected to the ...


In [ ]:
movies_metadata[movies_metadata['adult'] ==' Avalanche Sharks tells the story of a bikini contest that turns into a horrifying affair when it is hit by a shark avalanche.' ][['title', 'adult']].head()


,title,adult
35587,None,Avalanche Sharks tells the story of a bikini ...


In [ ]:
movies_metadata[movies_metadata['adult'] ==' - Written by Ørnås' ][['title', 'adult']].head()


,title,adult
19730,None,- Written by Ørnås


In [ ]:
movies_metadata[movies_metadata['adult'] =='True' ][['title', 'adult']].head(20)


,title,adult
19489,Erotic Nights of the Living Dead,True
28701,Standoff,True
31934,Electrical Girl,True
32113,Diet of Sex,True
39901,Amateur Porn Star Killer 2,True
39902,The Band,True
40574,The Sinful Dwarf,True
41009,Adulterers,True
43090,Half -Life,True


There are negligible number of adult movies in this dataset. The adult feature therefore is not of much use to us and can be safely dropped.

In [ ]:
movies_metadata.drop('adult',inplace=True,axis=1)
movies_metadata.head()

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,30000000,Animation|Comedy|Family,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0
1,65000000,Adventure|Fantasy|Family,8844,en,When siblings Judy and Peter discover an encha...,17.015539,TriStar Pictures|Teitler Film|Interscope Commu...,United States of America,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0
2,0,Romance|Comedy,15602,en,A family wedding reignites the ancient feud be...,11.7129,Warner Bros.|Lancaster Gate,United States of America,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0
3,16000000,Comedy|Drama|Romance,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,Twentieth Century Fox Film Corporation,United States of America,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0
4,0,Comedy,11862,en,Just when George Banks has recovered from his ...,8.387519,Sandollar Productions|Touchstone Pictures,United States of America,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0


Let's explore the dataset a bit more.

In [ ]:
movies_metadata['status'].value_counts()


Released           45014
Rumored              230
Post Production       98
In Production         20
Planned               15
Canceled               2
Name: status, dtype: int64

In [ ]:
movies_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                45466 non-null  object 
 1   genres                45466 non-null  object 
 2   id                    45466 non-null  object 
 3   original_language     45455 non-null  object 
 4   overview              44512 non-null  object 
 5   popularity            45461 non-null  object 
 6   production_companies  45466 non-null  object 
 7   production_countries  45466 non-null  object 
 8   release_date          45379 non-null  object 
 9   revenue               45460 non-null  float64
 10  runtime               45203 non-null  float64
 11  spoken_languages      45460 non-null  object 
 12  status                45379 non-null  object 
 13  tagline               20412 non-null  object 
 14  title                 45460 non-null  object 
 15  vote_average       

Since both the columns **'overview'** and **'tagline'** contains description of the movie...let's combine these two column and form a single column **'content'** .

In [ ]:
movies_metadata['overview'] = movies_metadata['overview'].fillna('')
movies_metadata['tagline'] = movies_metadata['tagline'].fillna('')
movies_metadata['content'] = movies_metadata['overview']+movies_metadata['tagline']
movies_metadata.drop('overview',inplace=True,axis=1)
movies_metadata.drop('tagline',inplace=True,axis=1)
movies_metadata.head(5)

,budget,genres,id,original_language,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,title,vote_average,vote_count,content
0,30000000,Animation|Comedy|Family,862,en,21.946943,Pixar Animation Studios,United States of America,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Toy Story,7.7,5415.0,"Led by Woody, Andy's toys live happily in his ..."
1,65000000,Adventure|Fantasy|Family,8844,en,17.015539,TriStar Pictures|Teitler Film|Interscope Commu...,United States of America,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Jumanji,6.9,2413.0,When siblings Judy and Peter discover an encha...
2,0,Romance|Comedy,15602,en,11.7129,Warner Bros.|Lancaster Gate,United States of America,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Grumpier Old Men,6.5,92.0,A family wedding reignites the ancient feud be...
3,16000000,Comedy|Drama|Romance,31357,en,3.859495,Twentieth Century Fox Film Corporation,United States of America,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Waiting to Exhale,6.1,34.0,"Cheated on, mistreated and stepped on, the wom..."
4,0,Comedy,11862,en,8.387519,Sandollar Productions|Touchstone Pictures,United States of America,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Father of the Bride Part II,5.7,173.0,Just when George Banks has recovered from his ...


Now let's count the number of movies with the **'content'** column empty.We may require this data while creating **Content-based Recommendation System**

In [ ]:
movies_metadata[movies_metadata['content']==''].id.count()

946

We are provided with the **release_date** of the movies but we really don't require the date as such but the **year** and **dayofweek** of release might be useful. So we extract them and store them in two new columns **'release_day'** and **'release_year'**

In [ ]:


movies_metadata['release_year'] = pd.to_datetime(movies_metadata['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)
movies_metadata['release_day'] = pd.to_datetime(movies_metadata['release_date'], errors='coerce').dt.day_name()
movies_metadata.drop('release_date',inplace=True,axis=1)
movies_metadata.head()



,budget,genres,id,original_language,popularity,production_companies,production_countries,revenue,runtime,spoken_languages,status,title,vote_average,vote_count,content,release_year,release_day
0,30000000,Animation|Comedy|Family,862,en,21.946943,Pixar Animation Studios,United States of America,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Toy Story,7.7,5415.0,"Led by Woody, Andy's toys live happily in his ...",1995,Monday
1,65000000,Adventure|Fantasy|Family,8844,en,17.015539,TriStar Pictures|Teitler Film|Interscope Commu...,United States of America,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Jumanji,6.9,2413.0,When siblings Judy and Peter discover an encha...,1995,Friday
2,0,Romance|Comedy,15602,en,11.7129,Warner Bros.|Lancaster Gate,United States of America,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Grumpier Old Men,6.5,92.0,A family wedding reignites the ancient feud be...,1995,Friday
3,16000000,Comedy|Drama|Romance,31357,en,3.859495,Twentieth Century Fox Film Corporation,United States of America,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Waiting to Exhale,6.1,34.0,"Cheated on, mistreated and stepped on, the wom...",1995,Friday
4,0,Comedy,11862,en,8.387519,Sandollar Productions|Touchstone Pictures,United States of America,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Father of the Bride Part II,5.7,173.0,Just when George Banks has recovered from his ...,1995,Friday


Let's now focus on the three columns of the dataset : **'popularity','vote_average','vote_count'** 

In [ ]:
movies_metadata['popularity'].describe()

count     45461
unique    43758
top         0.0
freq         66
Name: popularity, dtype: object

On trying to convert the dtype of **'popularity'** column from object to float.it was found that there is a faulty row.So the row is filtered and then the conversion was made successfully

In [ ]:
movies_metadata=movies_metadata[movies_metadata['popularity']!='Beware Of Frost Bites']
movies_metadata['popularity']=movies_metadata['popularity'].astype(float)
movies_metadata.head()


,budget,genres,id,original_language,popularity,production_companies,production_countries,revenue,runtime,spoken_languages,status,title,vote_average,vote_count,content,release_year,release_day
0,30000000,Animation|Comedy|Family,862,en,21.946943,Pixar Animation Studios,United States of America,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Toy Story,7.7,5415.0,"Led by Woody, Andy's toys live happily in his ...",1995,Monday
1,65000000,Adventure|Fantasy|Family,8844,en,17.015539,TriStar Pictures|Teitler Film|Interscope Commu...,United States of America,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Jumanji,6.9,2413.0,When siblings Judy and Peter discover an encha...,1995,Friday
2,0,Romance|Comedy,15602,en,11.712900,Warner Bros.|Lancaster Gate,United States of America,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Grumpier Old Men,6.5,92.0,A family wedding reignites the ancient feud be...,1995,Friday
3,16000000,Comedy|Drama|Romance,31357,en,3.859495,Twentieth Century Fox Film Corporation,United States of America,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Waiting to Exhale,6.1,34.0,"Cheated on, mistreated and stepped on, the wom...",1995,Friday
4,0,Comedy,11862,en,8.387519,Sandollar Productions|Touchstone Pictures,United States of America,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Father of the Bride Part II,5.7,173.0,Just when George Banks has recovered from his ...,1995,Friday


Now we replace the Nan values of **'popularity' , 'vote_average' and 'vote_count'** with the median of the available values and then sort the entire dataset in descending order of the three columns.

In [ ]:
popularity_list = movies_metadata[movies_metadata['popularity']!=np.nan]['popularity']
movies_metadata['popularity']=movies_metadata['popularity'].replace(np.nan,popularity_list.median())
voteavg_list = movies_metadata[movies_metadata['vote_average']!=np.nan]['vote_average']
movies_metadata['vote_average']=movies_metadata['vote_average'].replace(np.nan,voteavg_list.median())
votecnt_list = movies_metadata[movies_metadata['vote_count']!=np.nan]['vote_count']
movies_metadata['vote_count']=movies_metadata['vote_count'].replace(np.nan,votecnt_list.median())
movies_metadata.sort_values(by=['popularity','vote_average','vote_count'],inplace=True,axis=0,ascending=False)

In [ ]:
movies_metadata.head(20)

,budget,genres,id,original_language,popularity,production_companies,production_countries,revenue,runtime,spoken_languages,status,title,vote_average,vote_count,content,release_year,release_day
30700,74000000,Family|Animation|Adventure|Comedy,211672,en,547.488298,Universal Pictures|Illumination Entertainment,United States of America,1.156731e+09,91.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Minions,6.4,4729.0,"Minions Stuart, Kevin and Bob are recruited by...",2015,Wednesday
33356,149000000,Action|Adventure|Fantasy,297762,en,294.337037,Dune Entertainment|Atlas Entertainment|Warner ...,United States of America,8.205804e+08,141.0,"[{'iso_639_1': 'de', 'name': 'Deutsch'}, {'iso...",Released,Wonder Woman,7.2,5025.0,An Amazon princess comes to the world of Man t...,2017,Tuesday
42222,160000000,Family|Fantasy|Romance,321612,en,287.253654,Walt Disney Pictures|Mandeville Films,United Kingdom|United States of America,1.262886e+09,129.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Beauty and the Beast,6.8,5530.0,A live-action adaptation of Disney's version o...,2017,Thursday
43644,34000000,Action|Crime,339403,en,228.032744,Big Talk Productions|TriStar Pictures|Media Ri...,United Kingdom|United States of America,2.245113e+08,113.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Baby Driver,7.2,2083.0,After being coerced into working for a crime b...,2017,Wednesday
24455,165000000,Adventure|Family|Animation|Action|Comedy,177572,en,213.849907,Walt Disney Pictures|Walt Disney Animation Stu...,United States of America,6.521054e+08,102.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Big Hero 6,7.8,6289.0,The special bond that develops between plus-si...,2014,Friday
26564,58000000,Action|Adventure|Comedy,293660,en,187.860492,Twentieth Century Fox Film Corporation|Marvel ...,United States of America,7.831130e+08,108.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Deadpool,7.4,11444.0,Deadpool tells the origin story of former Spec...,2016,Tuesday
26566,200000000,Action|Adventure|Comedy|Science Fiction,283995,en,185.330992,Walt Disney Pictures|Marvel Studios,United States of America,8.634161e+08,137.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Guardians of the Galaxy Vol. 2,7.6,4858.0,The Guardians must fight to keep their newfoun...,2017,Wednesday
14551,237000000,Action|Adventure|Fantasy|Science Fiction,19995,en,185.070892,Ingenious Film Partners|Twentieth Century Fox ...,United States of America|United Kingdom,2.787965e+09,162.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Avatar,7.2,12114.0,"In the 22nd century, a paraplegic Marine is di...",2009,Thursday
24351,20000000,Action|Thriller,245891,en,183.870374,Summit Entertainment|Warner Bros.|87Eleven|Def...,Canada|China|United States of America,8.876166e+07,101.0,"[{'iso_639_1': 'hu', 'name': 'Magyar'}, {'iso_...",Released,John Wick,7.0,5499.0,Ex-lunatic John Wick comes off his meds to tra...,2014,Wednesday
23675,61000000,Mystery|Thriller|Drama,210577,en,154.801009,Twentieth Century Fox Film Corporation|Regency...,United States of America,3.693304e+08,145.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Gone Girl,7.9,6023.0,With his wife's disappearance having become th...,2014,Wednesday


**Resetting Index post Sorting**

In [ ]:
movies_metadata=movies_metadata.reset_index(drop=True)
movies_metadata.head()

,budget,genres,id,original_language,popularity,production_companies,production_countries,revenue,runtime,spoken_languages,status,title,vote_average,vote_count,content,release_year,release_day
0,74000000,Family|Animation|Adventure|Comedy,211672,en,547.488298,Universal Pictures|Illumination Entertainment,United States of America,1.156731e+09,91.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Minions,6.4,4729.0,"Minions Stuart, Kevin and Bob are recruited by...",2015,Wednesday
1,149000000,Action|Adventure|Fantasy,297762,en,294.337037,Dune Entertainment|Atlas Entertainment|Warner ...,United States of America,8.205804e+08,141.0,"[{'iso_639_1': 'de', 'name': 'Deutsch'}, {'iso...",Released,Wonder Woman,7.2,5025.0,An Amazon princess comes to the world of Man t...,2017,Tuesday
2,160000000,Family|Fantasy|Romance,321612,en,287.253654,Walt Disney Pictures|Mandeville Films,United Kingdom|United States of America,1.262886e+09,129.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Beauty and the Beast,6.8,5530.0,A live-action adaptation of Disney's version o...,2017,Thursday
3,34000000,Action|Crime,339403,en,228.032744,Big Talk Productions|TriStar Pictures|Media Ri...,United Kingdom|United States of America,2.245113e+08,113.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Baby Driver,7.2,2083.0,After being coerced into working for a crime b...,2017,Wednesday
4,165000000,Adventure|Family|Animation|Action|Comedy,177572,en,213.849907,Walt Disney Pictures|Walt Disney Animation Stu...,United States of America,6.521054e+08,102.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Big Hero 6,7.8,6289.0,The special bond that develops between plus-si...,2014,Friday


**Budget Column:** There are too many 0s in the 'budget' column which is absurd. Hence  the 0 values are replaced with median of the non-zero values of 'budget' column. Two rows with absurd value of budget is removed completely

In [ ]:
movies_metadata=movies_metadata[movies_metadata['budget']!='/ff9qCepilowshEtG2GYWwzt2bs4.jpg']
movies_metadata=movies_metadata[movies_metadata['budget']!='/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg']
movies_metadata['budget']=movies_metadata['budget'].astype(int)
budget_list = movies_metadata[movies_metadata['budget']!=0]['budget']
movies_metadata['budget']=movies_metadata['budget'].replace(0,budget_list.median())

In [ ]:
movies_metadata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45463 entries, 0 to 45464
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                45463 non-null  int64  
 1   genres                45463 non-null  object 
 2   id                    45463 non-null  object 
 3   original_language     45452 non-null  object 
 4   popularity            45463 non-null  float64
 5   production_companies  45463 non-null  object 
 6   production_countries  45463 non-null  object 
 7   revenue               45460 non-null  float64
 8   runtime               45203 non-null  float64
 9   spoken_languages      45460 non-null  object 
 10  status                45379 non-null  object 
 11  title                 45460 non-null  object 
 12  vote_average          45463 non-null  float64
 13  vote_count            45463 non-null  float64
 14  content               45463 non-null  object 
 15  release_year       

**Original_language column:**  The **'original_language'** column has language code as its value. The **'spoken_languages'** column has both language code and language name as values. Hence the language codes are replaced with language name in the **'original_language'** column with the help of the **'spoken_languages'** column. Then the **'spoken_languages'** column is dropped.

In [ ]:
movies_metadata['spoken_languages'] = movies_metadata['spoken_languages'].fillna('[]').apply(literal_eval).apply(lambda x: [(i['iso_639_1'],i['name']) for i in x] if isinstance(x, list) else [])
movies_metadata.head()

,budget,genres,id,original_language,popularity,production_companies,production_countries,revenue,runtime,spoken_languages,status,title,vote_average,vote_count,content,release_year,release_day
0,74000000,Family|Animation|Adventure|Comedy,211672,en,547.488298,Universal Pictures|Illumination Entertainment,United States of America,1.156731e+09,91.0,"[(en, English)]",Released,Minions,6.4,4729.0,"Minions Stuart, Kevin and Bob are recruited by...",2015,Wednesday
1,149000000,Action|Adventure|Fantasy,297762,en,294.337037,Dune Entertainment|Atlas Entertainment|Warner ...,United States of America,8.205804e+08,141.0,"[(de, Deutsch), (en, English)]",Released,Wonder Woman,7.2,5025.0,An Amazon princess comes to the world of Man t...,2017,Tuesday
2,160000000,Family|Fantasy|Romance,321612,en,287.253654,Walt Disney Pictures|Mandeville Films,United Kingdom|United States of America,1.262886e+09,129.0,"[(en, English)]",Released,Beauty and the Beast,6.8,5530.0,A live-action adaptation of Disney's version o...,2017,Thursday
3,34000000,Action|Crime,339403,en,228.032744,Big Talk Productions|TriStar Pictures|Media Ri...,United Kingdom|United States of America,2.245113e+08,113.0,"[(en, English)]",Released,Baby Driver,7.2,2083.0,After being coerced into working for a crime b...,2017,Wednesday
4,165000000,Adventure|Family|Animation|Action|Comedy,177572,en,213.849907,Walt Disney Pictures|Walt Disney Animation Stu...,United States of America,6.521054e+08,102.0,"[(en, English)]",Released,Big Hero 6,7.8,6289.0,The special bond that develops between plus-si...,2014,Friday


In [ ]:
languages={}
for index in movies_metadata.index:
  for item in movies_metadata.loc[index,'spoken_languages']:
    languages[item[0]]=item[1]
languages

{'ab': '',
 'af': 'Afrikaans',
 'am': '',
 'ar': 'العربية',
 'as': '',
 'ay': '',
 'az': 'Azərbaycan',
 'be': 'беларуская мова',
 'bg': 'български език',
 'bi': '',
 'bm': 'Bamanankan',
 'bn': 'বাংলা',
 'bo': '',
 'br': '',
 'bs': 'Bosanski',
 'ca': 'Català',
 'ce': '',
 'cn': '广州话 / 廣州話',
 'co': '',
 'cr': '',
 'cs': 'Český',
 'cy': 'Cymraeg',
 'da': 'Dansk',
 'de': 'Deutsch',
 'dz': '',
 'el': 'ελληνικά',
 'en': 'English',
 'eo': 'Esperanto',
 'es': 'Español',
 'et': 'Eesti',
 'eu': 'euskera',
 'fa': 'فارسی',
 'ff': 'Fulfulde',
 'fi': 'suomi',
 'fo': '',
 'fr': 'Français',
 'fy': '',
 'ga': 'Gaeilge',
 'gd': '',
 'gl': 'Galego',
 'gn': '',
 'gu': '',
 'ha': 'Hausa',
 'he': 'עִבְרִית',
 'hi': 'हिन्दी',
 'hr': 'Hrvatski',
 'ht': '',
 'hu': 'Magyar',
 'hy': '',
 'id': 'Bahasa indonesia',
 'ig': '',
 'is': 'Íslenska',
 'it': 'Italiano',
 'iu': '',
 'ja': '日本語',
 'jv': '',
 'ka': 'ქართული',
 'ki': '',
 'kk': 'қазақ',
 'km': '',
 'kn': '?????',
 'ko': '한국어/조선말',
 'ku': '',
 'kw': '',
 'ky'

In [ ]:
for index in movies_metadata.index:
  if movies_metadata.loc[index,'original_language'] in languages.keys():
    movies_metadata.loc[index,'original_language']=languages[movies_metadata.loc[index,'original_language']]
movies_metadata['original_language'].fillna('Unknown',inplace=True)
movies_metadata['original_language'].value_counts()

English        32269
Français        2438
Italiano        1529
日本語             1350
Deutsch         1080
               ...  
Latin              1
Galego             1
Esperanto          1
Kinyarwanda        1
Malti              1
Name: original_language, Length: 69, dtype: int64

**Filling Null Values Using Various Methods**

In [ ]:
movies_metadata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45463 entries, 0 to 45464
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                45463 non-null  int64  
 1   genres                45463 non-null  object 
 2   id                    45463 non-null  object 
 3   original_language     45463 non-null  object 
 4   popularity            45463 non-null  float64
 5   production_companies  45463 non-null  object 
 6   production_countries  45463 non-null  object 
 7   revenue               45460 non-null  float64
 8   runtime               45203 non-null  float64
 9   spoken_languages      45463 non-null  object 
 10  status                45379 non-null  object 
 11  title                 45460 non-null  object 
 12  vote_average          45463 non-null  float64
 13  vote_count            45463 non-null  float64
 14  content               45463 non-null  object 
 15  release_year       

In [ ]:
movies_metadata['status'].fillna('Unknown',inplace=True)
movies_metadata['title'].fillna('Unknown',inplace=True)
movies_metadata['release_day'].fillna('Unknown',inplace=True)
movies_metadata['revenue'].fillna(movies_metadata['revenue'].median(),inplace=True)
movies_metadata['runtime'].fillna(movies_metadata['runtime'].mean(),inplace=True)

In [ ]:
movies_metadata=movies_metadata.reset_index(drop=True)
movies_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45463 entries, 0 to 45462
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                45463 non-null  int64  
 1   genres                45463 non-null  object 
 2   id                    45463 non-null  object 
 3   original_language     45463 non-null  object 
 4   popularity            45463 non-null  float64
 5   production_companies  45463 non-null  object 
 6   production_countries  45463 non-null  object 
 7   revenue               45463 non-null  float64
 8   runtime               45463 non-null  float64
 9   spoken_languages      45463 non-null  object 
 10  status                45463 non-null  object 
 11  title                 45463 non-null  object 
 12  vote_average          45463 non-null  float64
 13  vote_count            45463 non-null  float64
 14  content               45463 non-null  object 
 15  release_year       

**Rearranging Column Names :**

In [ ]:
movies_metadata=movies_metadata[['id','title','release_year','release_day','genres','original_language','runtime','content','production_companies','budget','revenue','production_countries','status','popularity','vote_average','vote_count']]

**Top 5 Bengali Movies According to Popularity, Vote Counts And Vote Average :**

In [ ]:
movies_metadata[movies_metadata['original_language']=='বাংলা'].head()

,id,title,release_year,release_day,genres,original_language,runtime,content,production_companies,budget,revenue,production_countries,status,popularity,vote_average,vote_count
10969,5801,Pather Panchali,1955,Friday,Drama,বাংলা,125.0,The film is a coming-of-age story of a young b...,Government of West Bengal,8000000,536364.0,India,Released,3.900709,7.9,63.0
13188,167494,"Reason, Debate and a Story",1974,Tuesday,Drama,বাংলা,120.0,"Four men, each peculiar in his own way, embark...",,8000000,0.0,India,Released,2.876070,5.0,1.0
13376,897,Aparajito,1956,Thursday,Drama,বাংলা,110.0,The Unvanquished is the second part of the Ban...,Epic Productions,8000000,170215.0,India,Released,2.800510,7.9,44.0
14768,896,The World of Apu,1959,Friday,Drama,বাংলা,117.0,Apu is a jobless ex-student dreaming vaguely o...,Satyajit Ray Productions,8000000,16000.0,India,Released,2.381806,8.2,40.0
15323,165432,Joi Baba Felunath,1979,Thursday,,বাংলা,112.0,"Set in the holy city of Benares, this is the s...",,8000000,0.0,India,Released,2.238456,7.3,5.0


**Top 5 Hindi Movies According to Popularity, Vote Counts And Vote Average :**

In [ ]:
movies_metadata[movies_metadata['original_language']=='हिन्दी'].head()

,id,title,release_year,release_day,genres,original_language,runtime,content,production_companies,budget,revenue,production_countries,status,popularity,vote_average,vote_count
79,19404,Dilwale Dulhania Le Jayenge,1995,Friday,Comedy|Drama|Romance,हिन्दी,190.0,"Raj is a rich, carefree, happy-go-lucky second...",Yash Raj Films,13200000,100000000.0,India,Released,34.457024,9.1,661.0
1445,297222,PK,2014,Thursday,Comedy|Drama|Science Fiction,हिन्दी,153.0,This experimental satire is a comedy of ideas ...,Rajkumar Hirani Films|Vinod Chopra Films,13000000,120000000.0,India,Released,12.587520,7.7,262.0
1602,14073,Jodhaa Akbar,2008,Friday,Action|Adventure|Drama|History|Romance,हिन्दी,213.0,Jodhaa Akbar is a sixteenth century love story...,UTV Motion Pictures|Ashutosh Gowariker Product...,8376800,13000000.0,India,Released,12.237488,7.6,74.0
2130,26022,My Name Is Khan,2010,Friday,Drama|Romance,हिन्दी,145.0,"Rizwan Khan, a Muslim from the Borivali sectio...",Fox Searchlight Pictures|Red Chillies Entertai...,12000000,42345360.0,India|United States of America,Released,11.264586,7.7,237.0
2225,204435,Krrish 3,2013,Friday,Action|Science Fiction,हिन्दी,152.0,After defeating the villainous Dr. Siddhant Ar...,,6218100,0.0,India,Released,11.114030,5.8,42.0


**IMPORTING links.csv :** This dataset contains the TMDB and IMDB ids of the movies in **'movies_metadata.csv'** .

In [ ]:
link2='https://drive.google.com/file/d/1ZSyxmhiBnJNT7zEng7ak6_SCms9Y6RPp/view?usp=sharing'

In [ ]:
id='1ZSyxmhiBnJNT7zEng7ak6_SCms9Y6RPp'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('links.csv')  
links = pd.read_csv('links.csv',engine='python')
links.head(5)

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [ ]:
links.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45843 entries, 0 to 45842
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  45843 non-null  int64  
 1   imdbId   45843 non-null  int64  
 2   tmdbId   45624 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 1.0 MB


**MERGING THE 'movies_metadata' DATAFRAME WITH THE 'links.csv'**

In [ ]:
movies_metadata['id'] = pd.to_numeric(movies_metadata['id'])
movies_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45463 entries, 0 to 45462
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    45463 non-null  int64  
 1   title                 45463 non-null  object 
 2   release_year          45463 non-null  object 
 3   release_day           45463 non-null  object 
 4   genres                45463 non-null  object 
 5   original_language     45463 non-null  object 
 6   runtime               45463 non-null  float64
 7   content               45463 non-null  object 
 8   production_companies  45463 non-null  object 
 9   budget                45463 non-null  int64  
 10  revenue               45463 non-null  float64
 11  production_countries  45463 non-null  object 
 12  status                45463 non-null  object 
 13  popularity            45463 non-null  float64
 14  vote_average          45463 non-null  float64
 15  vote_count         

In [ ]:
links['tmdbId']=np.array(links['tmdbId'], np.int64)
links.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45843 entries, 0 to 45842
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   movieId  45843 non-null  int64
 1   imdbId   45843 non-null  int64
 2   tmdbId   45843 non-null  int64
dtypes: int64(3)
memory usage: 1.0 MB


In [ ]:
movies_metadata.rename(columns={'id':'tmdbId'}, inplace=True)
movies_data = pd.merge(movies_metadata,links,on='tmdbId')
movies_data.head()

,tmdbId,title,release_year,release_day,genres,original_language,runtime,content,production_companies,budget,revenue,production_countries,status,popularity,vote_average,vote_count,movieId,imdbId
0,211672,Minions,2015,Wednesday,Family|Animation|Adventure|Comedy,English,91.0,"Minions Stuart, Kevin and Bob are recruited by...",Universal Pictures|Illumination Entertainment,74000000,1.156731e+09,United States of America,Released,547.488298,6.4,4729.0,135887,2293640
1,297762,Wonder Woman,2017,Tuesday,Action|Adventure|Fantasy,English,141.0,An Amazon princess comes to the world of Man t...,Dune Entertainment|Atlas Entertainment|Warner ...,149000000,8.205804e+08,United States of America,Released,294.337037,7.2,5025.0,143355,451279
2,321612,Beauty and the Beast,2017,Thursday,Family|Fantasy|Romance,English,129.0,A live-action adaptation of Disney's version o...,Walt Disney Pictures|Mandeville Films,160000000,1.262886e+09,United Kingdom|United States of America,Released,287.253654,6.8,5530.0,168366,2771200
3,339403,Baby Driver,2017,Wednesday,Action|Crime,English,113.0,After being coerced into working for a crime b...,Big Talk Productions|TriStar Pictures|Media Ri...,34000000,2.245113e+08,United Kingdom|United States of America,Released,228.032744,7.2,2083.0,171763,3890160
4,177572,Big Hero 6,2014,Friday,Adventure|Family|Animation|Action|Comedy,English,102.0,The special bond that develops between plus-si...,Walt Disney Pictures|Walt Disney Animation Stu...,165000000,6.521054e+08,United States of America,Released,213.849907,7.8,6289.0,115617,2245084


In [ ]:
movies_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45525 entries, 0 to 45524
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   tmdbId                45525 non-null  int64  
 1   title                 45525 non-null  object 
 2   release_year          45525 non-null  object 
 3   release_day           45525 non-null  object 
 4   genres                45525 non-null  object 
 5   original_language     45525 non-null  object 
 6   runtime               45525 non-null  float64
 7   content               45525 non-null  object 
 8   production_companies  45525 non-null  object 
 9   budget                45525 non-null  int64  
 10  revenue               45525 non-null  float64
 11  production_countries  45525 non-null  object 
 12  status                45525 non-null  object 
 13  popularity            45525 non-null  float64
 14  vote_average          45525 non-null  float64
 15  vote_count         

**REMOVING ROWS WITH UNKNOWN TITLE (MOVIE NAME):**

In [ ]:
movies_data[movies_data['title']=="Unknown"].tmdbId.count()

5

In [ ]:
movies_data=movies_data[movies_data['title']!="Unknown"]
movies_data.reset_index(drop=True)
movies_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45520 entries, 0 to 45524
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   tmdbId                45520 non-null  int64  
 1   title                 45520 non-null  object 
 2   release_year          45520 non-null  object 
 3   release_day           45520 non-null  object 
 4   genres                45520 non-null  object 
 5   original_language     45520 non-null  object 
 6   runtime               45520 non-null  float64
 7   content               45520 non-null  object 
 8   production_companies  45520 non-null  object 
 9   budget                45520 non-null  int64  
 10  revenue               45520 non-null  float64
 11  production_countries  45520 non-null  object 
 12  status                45520 non-null  object 
 13  popularity            45520 non-null  float64
 14  vote_average          45520 non-null  float64
 15  vote_count         

**IMPORTING 'credits.csv'**

In [ ]:
link3='https://drive.google.com/file/d/1zucIWQqlWHXxwyfO1lc_hleV76RbYwGa/view?usp=sharing'

In [ ]:
id='1zucIWQqlWHXxwyfO1lc_hleV76RbYwGa'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('credits.csv')  
credits = pd.read_csv('credits.csv',engine='python')
credits.head(5)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


**EXTRACTING RELEVANT INFORMATION :** The actors' names from the 'cast' column and the director's name from the crew is extracted.

In [ ]:
credits['cast'] = credits['cast'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
credits['cast']=[try_join(l)for l in credits['cast']]

In [ ]:
credits.head(10)

,cast,crew,id
0,Tom Hanks|Tim Allen|Don Rickles|Jim Varney|Wal...,"[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,Robin Williams|Jonathan Hyde|Kirsten Dunst|Bra...,"[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,Walter Matthau|Jack Lemmon|Ann-Margret|Sophia ...,"[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,Whitney Houston|Angela Bassett|Loretta Devine|...,"[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,Steve Martin|Diane Keaton|Martin Short|Kimberl...,"[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
5,Al Pacino|Robert De Niro|Val Kilmer|Jon Voight...,"[{'credit_id': '52fe4292c3a36847f802916d', 'de...",949
6,Harrison Ford|Julia Ormond|Greg Kinnear|Angie ...,"[{'credit_id': '52fe44959251416c75039da9', 'de...",11860
7,Jonathan Taylor Thomas|Brad Renfro|Rachael Lei...,"[{'credit_id': '52fe46bdc3a36847f810f797', 'de...",45325
8,Jean-Claude Van Damme|Powers Boothe|Dorian Har...,"[{'credit_id': '52fe44dbc3a36847f80ae0f1', 'de...",9091
9,Pierce Brosnan|Sean Bean|Izabella Scorupco|Fam...,"[{'credit_id': '52fe426ec3a36847f801e14b', 'de...",710


In [ ]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return 'Unknown'

credits['director'] =credits['crew'].apply(literal_eval).apply(get_director)
credits.drop('crew',axis=1,inplace=True)
credits.head()

,cast,id,director
0,Tom Hanks|Tim Allen|Don Rickles|Jim Varney|Wal...,862,John Lasseter
1,Robin Williams|Jonathan Hyde|Kirsten Dunst|Bra...,8844,Joe Johnston
2,Walter Matthau|Jack Lemmon|Ann-Margret|Sophia ...,15602,Howard Deutch
3,Whitney Houston|Angela Bassett|Loretta Devine|...,31357,Forest Whitaker
4,Steve Martin|Diane Keaton|Martin Short|Kimberl...,11862,Charles Shyer


In [ ]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   cast      45476 non-null  object
 1   id        45476 non-null  int64 
 2   director  45476 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


**MERGING THE DATAFRAMES 'movies_data' and 'credits' :**

In [ ]:
credits.rename(columns={'id':'tmdbId'}, inplace=True)
movies_data = pd.merge(movies_data,credits,on='tmdbId')
movies_data.head()

,tmdbId,title,release_year,release_day,genres,original_language,runtime,content,production_companies,budget,revenue,production_countries,status,popularity,vote_average,vote_count,movieId,imdbId,cast,director
0,211672,Minions,2015,Wednesday,Family|Animation|Adventure|Comedy,English,91.0,"Minions Stuart, Kevin and Bob are recruited by...",Universal Pictures|Illumination Entertainment,74000000,1.156731e+09,United States of America,Released,547.488298,6.4,4729.0,135887,2293640,Sandra Bullock|Jon Hamm|Michael Keaton|Allison...,Kyle Balda
1,297762,Wonder Woman,2017,Tuesday,Action|Adventure|Fantasy,English,141.0,An Amazon princess comes to the world of Man t...,Dune Entertainment|Atlas Entertainment|Warner ...,149000000,8.205804e+08,United States of America,Released,294.337037,7.2,5025.0,143355,451279,Gal Gadot|Chris Pine|Robin Wright|Danny Huston...,Patty Jenkins
2,321612,Beauty and the Beast,2017,Thursday,Family|Fantasy|Romance,English,129.0,A live-action adaptation of Disney's version o...,Walt Disney Pictures|Mandeville Films,160000000,1.262886e+09,United Kingdom|United States of America,Released,287.253654,6.8,5530.0,168366,2771200,Emma Watson|Dan Stevens|Luke Evans|Kevin Kline...,Bill Condon
3,339403,Baby Driver,2017,Wednesday,Action|Crime,English,113.0,After being coerced into working for a crime b...,Big Talk Productions|TriStar Pictures|Media Ri...,34000000,2.245113e+08,United Kingdom|United States of America,Released,228.032744,7.2,2083.0,171763,3890160,Ansel Elgort|Lily James|Kevin Spacey|Jamie Fox...,Edgar Wright
4,177572,Big Hero 6,2014,Friday,Adventure|Family|Animation|Action|Comedy,English,102.0,The special bond that develops between plus-si...,Walt Disney Pictures|Walt Disney Animation Stu...,165000000,6.521054e+08,United States of America,Released,213.849907,7.8,6289.0,115617,2245084,Scott Adsit|Ryan Potter|Daniel Henney|T.J. Mil...,Chris Williams


In [ ]:
movies_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45663 entries, 0 to 45662
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   tmdbId                45663 non-null  int64  
 1   title                 45663 non-null  object 
 2   release_year          45663 non-null  object 
 3   release_day           45663 non-null  object 
 4   genres                45663 non-null  object 
 5   original_language     45663 non-null  object 
 6   runtime               45663 non-null  float64
 7   content               45663 non-null  object 
 8   production_companies  45663 non-null  object 
 9   budget                45663 non-null  int64  
 10  revenue               45663 non-null  float64
 11  production_countries  45663 non-null  object 
 12  status                45663 non-null  object 
 13  popularity            45663 non-null  float64
 14  vote_average          45663 non-null  float64
 15  vote_count         

**Top 5 Hrithik Roshan Movies Based on Popularity, vote counts and vote average:**

In [ ]:
HRmovie = movies_data['cast'].str.contains('Hrithik Roshan')
movies_data[HRmovie].head()	

,tmdbId,title,release_year,release_day,genres,original_language,runtime,content,production_companies,budget,revenue,production_countries,status,popularity,vote_average,vote_count,movieId,imdbId,cast,director
1600,14073,Jodhaa Akbar,2008,Friday,Action|Adventure|Drama|History|Romance,हिन्दी,213.0,Jodhaa Akbar is a sixteenth century love story...,UTV Motion Pictures|Ashutosh Gowariker Product...,8376800,13000000.0,India,Released,12.237488,7.6,74.0,58111,449994,Hrithik Roshan|Aishwarya Rai Bachchan|Sonu Soo...,Ashutosh Gowariker
2243,204435,Krrish 3,2013,Friday,Action|Science Fiction,हिन्दी,152.0,After defeating the villainous Dr. Siddhant Ar...,,6218100,0.0,India,Released,11.114030,5.8,42.0,106561,1029231,Hrithik Roshan|Priyanka Chopra|Rajpal Yadav|Ka...,Rakesh Roshan
4410,61202,Zindagi Na Milegi Dobara,2011,Friday,Drama|Comedy|Romance,हिन्दी,155.0,"Mumbai-based Kabir Dhiman, who comes from a we...",Excel Entertainment|Eros International,8000000,0.0,India,Released,8.475505,7.6,83.0,89554,1562872,Hrithik Roshan|Farhan Akhtar|Abhay Deol|Katrin...,Zoya Akhtar
5651,403867,Kaabil,2017,Thursday,Action|Crime|Romance,हिन्दी,139.0,"Rohan (Hritik roshan) is a kind, happy young m...",Film Kraft,5200000,0.0,India,Released,7.404544,8.1,23.0,169888,5460276,Hrithik Roshan|Yami Gautam|Ronit Roy|Rohit Roy...,Sanjay Gupta
6849,10757,Kabhi Khushi Kabhie Gham,2001,Friday,Comedy|Drama,हिन्दी,210.0,"Orthodox Indian, Raichand, would like his two ...",,8000000,0.0,India,Released,6.523888,7.6,103.0,27362,248126,Shah Rukh Khan|Kajol|Amitabh Bachchan|Jaya Bac...,Karan Johar


**IMPORTING Ratings.csv**

In [ ]:
link4='https://drive.google.com/file/d/1ZkhkhHnDsy3v-9RhMAAJdLo1y7flVwe1/view?usp=sharing'

In [ ]:
id="1ZkhkhHnDsy3v-9RhMAAJdLo1y7flVwe1"
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('ratings.csv')  
ratings = pd.read_csv('ratings.csv',engine='python')
ratings.head(5)

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


The **'timestamp'** column seems irrelevant. So it is dropped

In [ ]:
ratings.drop('timestamp',axis=1,inplace=True)
ratings.head()

,userId,movieId,rating
0,1,110,1.0
1,1,147,4.5
2,1,858,5.0
3,1,1221,5.0
4,1,1246,5.0


In [ ]:
avg_ratings=ratings.groupby('movieId',as_index=False)['rating'].mean()
avg_ratings.head()

,movieId,rating
0,1,3.888157
1,2,3.236953
2,3,3.175550
3,4,2.875713
4,5,3.079565


In [ ]:
avg_ratings.isna().sum()

movieId    0
rating     0
dtype: int64

In [ ]:
movies_data = pd.merge(movies_data,avg_ratings,on='movieId')
movies_data.head()

,tmdbId,title,release_year,release_day,genres,original_language,runtime,content,production_companies,budget,revenue,production_countries,status,popularity,vote_average,vote_count,movieId,imdbId,cast,director,rating
0,211672,Minions,2015,Wednesday,Family|Animation|Adventure|Comedy,English,91.0,"Minions Stuart, Kevin and Bob are recruited by...",Universal Pictures|Illumination Entertainment,74000000,1.156731e+09,United States of America,Released,547.488298,6.4,4729.0,135887,2293640,Sandra Bullock|Jon Hamm|Michael Keaton|Allison...,Kyle Balda,3.135554
1,297762,Wonder Woman,2017,Tuesday,Action|Adventure|Fantasy,English,141.0,An Amazon princess comes to the world of Man t...,Dune Entertainment|Atlas Entertainment|Warner ...,149000000,8.205804e+08,United States of America,Released,294.337037,7.2,5025.0,143355,451279,Gal Gadot|Chris Pine|Robin Wright|Danny Huston...,Patty Jenkins,3.747344
2,321612,Beauty and the Beast,2017,Thursday,Family|Fantasy|Romance,English,129.0,A live-action adaptation of Disney's version o...,Walt Disney Pictures|Mandeville Films,160000000,1.262886e+09,United Kingdom|United States of America,Released,287.253654,6.8,5530.0,168366,2771200,Emma Watson|Dan Stevens|Luke Evans|Kevin Kline...,Bill Condon,3.585165
3,339403,Baby Driver,2017,Wednesday,Action|Crime,English,113.0,After being coerced into working for a crime b...,Big Talk Productions|TriStar Pictures|Media Ri...,34000000,2.245113e+08,United Kingdom|United States of America,Released,228.032744,7.2,2083.0,171763,3890160,Ansel Elgort|Lily James|Kevin Spacey|Jamie Fox...,Edgar Wright,3.973244
4,177572,Big Hero 6,2014,Friday,Adventure|Family|Animation|Action|Comedy,English,102.0,The special bond that develops between plus-si...,Walt Disney Pictures|Walt Disney Animation Stu...,165000000,6.521054e+08,United States of America,Released,213.849907,7.8,6289.0,115617,2245084,Scott Adsit|Ryan Potter|Daniel Henney|T.J. Mil...,Chris Williams,3.916242


In [ ]:
movies_data=movies_data.rename(columns={"rating":"mean_rating"})
movies_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44941 entries, 0 to 44940
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   tmdbId                44941 non-null  int64  
 1   title                 44941 non-null  object 
 2   release_year          44941 non-null  object 
 3   release_day           44941 non-null  object 
 4   genres                44941 non-null  object 
 5   original_language     44941 non-null  object 
 6   runtime               44941 non-null  float64
 7   content               44941 non-null  object 
 8   production_companies  44941 non-null  object 
 9   budget                44941 non-null  int64  
 10  revenue               44941 non-null  float64
 11  production_countries  44941 non-null  object 
 12  status                44941 non-null  object 
 13  popularity            44941 non-null  float64
 14  vote_average          44941 non-null  float64
 15  vote_count         

In [ ]:
movie_user_ratings_data=movies_data.merge(ratings,on = 'movieId',how = 'inner')
movie_user_ratings_data.head(40)

,tmdbId,title,release_year,release_day,genres,original_language,runtime,content,production_companies,budget,revenue,production_countries,status,popularity,vote_average,vote_count,movieId,imdbId,cast,director,mean_rating,userId,rating
0,211672,Minions,2015,Wednesday,Family|Animation|Adventure|Comedy,English,91.0,"Minions Stuart, Kevin and Bob are recruited by...",Universal Pictures|Illumination Entertainment,74000000,1.156731e+09,United States of America,Released,547.488298,6.4,4729.0,135887,2293640,Sandra Bullock|Jon Hamm|Michael Keaton|Allison...,Kyle Balda,3.135554,420,5.0
1,211672,Minions,2015,Wednesday,Family|Animation|Adventure|Comedy,English,91.0,"Minions Stuart, Kevin and Bob are recruited by...",Universal Pictures|Illumination Entertainment,74000000,1.156731e+09,United States of America,Released,547.488298,6.4,4729.0,135887,2293640,Sandra Bullock|Jon Hamm|Michael Keaton|Allison...,Kyle Balda,3.135554,505,1.5
2,211672,Minions,2015,Wednesday,Family|Animation|Adventure|Comedy,English,91.0,"Minions Stuart, Kevin and Bob are recruited by...",Universal Pictures|Illumination Entertainment,74000000,1.156731e+09,United States of America,Released,547.488298,6.4,4729.0,135887,2293640,Sandra Bullock|Jon Hamm|Michael Keaton|Allison...,Kyle Balda,3.135554,509,4.0
3,211672,Minions,2015,Wednesday,Family|Animation|Adventure|Comedy,English,91.0,"Minions Stuart, Kevin and Bob are recruited by...",Universal Pictures|Illumination Entertainment,74000000,1.156731e+09,United States of America,Released,547.488298,6.4,4729.0,135887,2293640,Sandra Bullock|Jon Hamm|Michael Keaton|Allison...,Kyle Balda,3.135554,559,4.0
4,211672,Minions,2015,Wednesday,Family|Animation|Adventure|Comedy,English,91.0,"Minions Stuart, Kevin and Bob are recruited by...",Universal Pictures|Illumination Entertainment,74000000,1.156731e+09,United States of America,Released,547.488298,6.4,4729.0,135887,2293640,Sandra Bullock|Jon Hamm|Michael Keaton|Allison...,Kyle Balda,3.135554,702,3.0
5,211672,Minions,2015,Wednesday,Family|Animation|Adventure|Comedy,English,91.0,"Minions Stuart, Kevin and Bob are recruited by...",Universal Pictures|Illumination Entertainment,74000000,1.156731e+09,United States of America,Released,547.488298,6.4,4729.0,135887,2293640,Sandra Bullock|Jon Hamm|Michael Keaton|Allison...,Kyle Balda,3.135554,836,3.0
6,211672,Minions,2015,Wednesday,Family|Animation|Adventure|Comedy,English,91.0,"Minions Stuart, Kevin and Bob are recruited by...",Universal Pictures|Illumination Entertainment,74000000,1.156731e+09,United States of America,Released,547.488298,6.4,4729.0,135887,2293640,Sandra Bullock|Jon Hamm|Michael Keaton|Allison...,Kyle Balda,3.135554,930,3.0
7,211672,Minions,2015,Wednesday,Family|Animation|Adventure|Comedy,English,91.0,"Minions Stuart, Kevin and Bob are recruited by...",Universal Pictures|Illumination Entertainment,74000000,1.156731e+09,United States of America,Released,547.488298,6.4,4729.0,135887,2293640,Sandra Bullock|Jon Hamm|Michael Keaton|Allison...,Kyle Balda,3.135554,1104,4.0
8,211672,Minions,2015,Wednesday,Family|Animation|Adventure|Comedy,English,91.0,"Minions Stuart, Kevin and Bob are recruited by...",Universal Pictures|Illumination Entertainment,74000000,1.156731e+09,United States of America,Released,547.488298,6.4,4729.0,135887,2293640,Sandra Bullock|Jon Hamm|Michael Keaton|Allison...,Kyle Balda,3.135554,1243,4.0
9,211672,Minions,2015,Wednesday,Family|Animation|Adventure|Comedy,English,91.0,"Minions Stuart, Kevin and Bob are recruited by...",Universal Pictures|Illumination Entertainment,74000000,1.156731e+09,United States of America,Released,547.488298,6.4,4729.0,135887,2293640,Sandra Bullock|Jon Hamm|Michael Keaton|Allison...,Kyle Balda,3.135554,1261,2.5


In [ ]:
movie_user_ratings_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26008196 entries, 0 to 26008195
Data columns (total 23 columns):
 #   Column                Dtype  
---  ------                -----  
 0   tmdbId                int64  
 1   title                 object 
 2   release_year          object 
 3   release_day           object 
 4   genres                object 
 5   original_language     object 
 6   runtime               float64
 7   content               object 
 8   production_companies  object 
 9   budget                int64  
 10  revenue               float64
 11  production_countries  object 
 12  status                object 
 13  popularity            float64
 14  vote_average          float64
 15  vote_count            float64
 16  movieId               int64  
 17  imdbId                int64  
 18  cast                  object 
 19  director              object 
 20  mean_rating           float64
 21  userId                int64  
 22  rating                float64
dtypes: fl